In [42]:
from collections import Counter
import json
from random import shuffle
sahil = None
with open('./data/vrckid.data', 'r') as f:
    sahil = f.read()
    sahil = json.loads(sahil)
others = []
for user in ['_vargas_.data', '-eDgaR-.data', 'Abe_lincolin.data',
             'Ambiguously_Ironic.data', 'anutensil.data', 'APOSTOLATE.data',
            'awildsketchappeared.data', 'axolotl_peyotl.data','boib.data']:
    with open('./data/' + user, 'r') as f:
        other = f.read()
        other = json.loads(other)
        shuffle(other)
        others.extend(other[:120])

In [43]:
import nltk
tokenizer = nltk.tokenize.casual.TweetTokenizer()

In [44]:
sahil_pos = []
for comment in sahil:
    tokens = tokenizer.tokenize(comment)
    pos = nltk.pos_tag(tokens)
    sahil_pos.append([tag for w, tag in pos])

In [45]:
others_pos = []
for comment in others:
    tokens = tokenizer.tokenize(comment)
    pos = nltk.pos_tag(tokens)
    others_pos.append([tag for w, tag in pos])

In [46]:
sahil_pos_bigram_counters = Counter()
for pos in sahil_pos:
    for bg in nltk.bigrams(pos):
        sahil_pos_bigram_counters[bg] += 1
#other_pos_bigram_counters = Counter()

In [65]:
feature_set = {pos for pos, freq in sahil_pos_bigram_counters.most_common(500)}

In [60]:
training_sahil_data = []
for pos in sahil_pos:
    row = [0] * (len(feature_set) + 1)
    pos_set = set(nltk.bigrams(pos))
    for i, pos_tuple in enumerate(feature_set):
        if pos_tuple in pos_set:
            row[i] = 1
    row[-1] = 1
    training_sahil_data.append(row)

training_other_data = []
for pos in others_pos:
    row = [0] * (len(feature_set) + 1)
    pos_set = set(nltk.bigrams(pos))
    for i, pos_tuple in enumerate(feature_set):
        if pos_tuple in pos_set:
            row[i] = 1
    training_other_data.append(row)

In [61]:
print sum([ sum(d) - 1 for d in training_sahil_data])
print sum([ sum(d) for d in training_other_data])

18604
24412


In [62]:
from random import shuffle
shuffle(training_sahil_data)
shuffle(training_other_data)
training = training_sahil_data[:int(len(training_sahil_data) * 0.8)] + training_other_data[:int(len(training_other_data) * 0.8)]
testing = training_sahil_data[int(len(training_sahil_data) * 0.8):] + training_other_data[int(len(training_other_data) * 0.8):]
shuffle(training)
shuffle(testing)
X = [row[:-1] for row in training]
Y = [row[-1] for row in training]

In [63]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()
logreg.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [64]:
test_X = [row[:-1] for row in testing]
test_Y = [row[-1] for row in testing]
correct_count = 0
for row, expected in zip(test_X, test_Y):
    predicted = logreg.predict(row)
    if predicted == expected:
        correct_count += 1
print 'Accuracy=', correct_count /float(len(test_X))

Accuracy= 0.674698795181


/Users/Juliang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Juliang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/Juliang/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single s